In [ ]:
import pyomo.environ as pe
from idaes.core import FlowsheetBlock, StateBlock
from idaes.models.unit_models import HeatExchanger
from idaes.models.unit_models import HeatExchanger1D
from idaes.models.unit_models.heat_exchanger import delta_temperature_amtd_callback
from idaes.models.properties import iapws95
import math

In [45]:
# Create an empty flowsheet and steam property parameter block
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(dynamic=False)
model.fs.properties = iapws95.Iapws95ParameterBlock()

In [51]:
# Import necessary libraries
import pyomo.environ as pyo
from idaes.models.unit_models import HeatExchanger1D
from idaes.core import FlowsheetBlock
from idaes.core.solvers import get_solver
from idaes.models.properties import iapws95

# Create the flowsheet block
model = FlowsheetBlock(default={"dynamic": False})

# Add the HeatExchanger1D unit operation
model.add_component("HX", HeatExchanger1D(default={
    "property_package": iapws95,
    "tube_side": {"flow_type": "counter"},
    "shell_side": {"flow_type": "counter"}
}))

In [1]:
# Load the area variable that was stored in HE_AreaCalculation.ipynb
%store -r A

# Now you can use the area variable
print(A)
model.fs.heat_exchanger.area.fix(A)

26.971917334619782


NameError: name 'model' is not defined

In [21]:
%store -r h_cold_inlet
%store  -r h_cold_outlet
%store  -r h_hot_inlet
%store  -r h_hot_outlet

In [ ]:

model.fs.heat_exchanger.shell_inlet.enth_mol.fix(h_cold_inlet)
model.fs.heat_exchanger.tube_inlet.enth_mol.fix(h_hot_inlet)

In [24]:
# Fix the parameters of the heat exchanger (inlets, pressures, and enthalpies)
model.fs.heat_exchanger.overall_heat_transfer_coefficient[0].fix(100)
model.fs.heat_exchanger.shell_inlet.pressure.fix(101325)
model.fs.heat_exchanger.shell_inlet.flow_mol.fix(10)
model.fs.heat_exchanger.tube_inlet.flow_mol.fix(10)
model.fs.heat_exchanger.tube_inlet.pressure.fix(101325)


In [27]:
# Create a variable to minimize the heat exchanger area
model.fs.heat_exchanger_area = pe.Var(within=pe.NonNegativeReals, initialize=A)

(type=<class 'pyomo.core.base.var.ScalarVar'>) on block fs with a new
Component (type=<class 'pyomo.core.base.var.AbstractScalarVar'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [28]:
# Assume the heat duty is already calculated or predefined (simplified for this example)
# Heat duty calculation: Hot side = Cold side (m_dot_h * (h_h_in - h_h_out) = m_dot_c * (h_c_out - h_c_in))
m_dot_h = model.fs.heat_exchanger.shell_inlet.flow_mol
m_dot_c = model.fs.heat_exchanger.tube_inlet.flow_mol

In [29]:
# Hot side inlet and outlet enthalpies
h_h_in = model.fs.heat_exchanger.shell_inlet.enth_mol
h_h_out = model.fs.heat_exchanger.shell_outlet.enth_mol

# Cold side inlet and outlet enthalpies
h_c_in = model.fs.heat_exchanger.tube_inlet.enth_mol
h_c_out = model.fs.heat_exchanger.tube_outlet.enth_mol

In [30]:

# Define area as a decision variable to be optimized
model.fs.heat_exchanger.area = pe.Var(within=pe.NonNegativeReals)

'pyomo.core.base.var.ScalarVar'>) on block fs.heat_exchanger with a new
Component (type=<class 'pyomo.core.base.var.AbstractScalarVar'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [31]:
# Set the objective to minimize the heat exchanger area
model.obj = pe.Objective(expr=model.fs.heat_exchanger_area)

'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [32]:

# Initialize the model
from idaes.models.unit_models.heat_exchanger import HX0DInitializer
initializer = HX0DInitializer()
initializer.initialize(model.fs.heat_exchanger)

2025-04-07 16:29:45 [INFO] idaes.init.fs.heat_exchanger: Initialization Completed, optimal - <undefined>


<InitializationStatus.Ok: 1>

In [33]:
# Solve the model using a solver (e.g., IPOPT)
solver = pe.SolverFactory('ipopt')
solver.solve(model, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 10, 'Number of variables': 11, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.1304616928100586}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [35]:
# Output the results
print(f"Optimized Heat Exchanger Area: {pe.value(model.fs.heat_exchanger_area)} m^2")
# Calculate the heat duty based on the hot side
heat_duty = pe.value(m_dot_h[0] * (h_h_in[0] - h_h_out[0]))
print(f"Optimized Heat Duty: {heat_duty/1000} kJ/mol")

Optimized Heat Exchanger Area: 0.0 m^2
Optimized Heat Duty: -6.630040546265966 kJ/mol


In [36]:
import idaes_ui
import idaes_ui.fv
dir(idaes_ui.fv)
from idaes_ui.fv import visualize 

# Assign a value to the heat exchanger area before visualization
model.fs.heat_exchanger.area.set_value(pe.value(model.fs.heat_exchanger_area))

# Visualize the model
visualize(model.fs, "HeatExchangerAreaMinimization")

2025-04-07 16:29:56 [INFO] idaes.idaes_ui.fv.fsvis: Using HTTP server on localhost, port 56445
2025-04-07 16:29:56 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'HeatExchangerAreaMinimization.json'
2025-04-07 16:29:56 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'HeatExchangerAreaMinimization.json' in current directory (c:\Users\Sara\Desktop\IDAES\HE)
2025-04-07 16:29:56 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:56445/app?id=HeatExchangerAreaMinimization


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x0000028C5F6B6650>, port=56445, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x0000028C5F6B5E40>)